![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2019/master/logo.png)

![Plan zajęć](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2019/master/plan_zajec.jpg)

# SELECT Python
## SQLite w Pythonie
### Wojciech Łuszczyński
#### Python level UP 01.04.2019

# Baza danych
- zbiór danych zapisanych zgodnie z ustalonymi regułami

# Systemy zarządzania bazą danych (DBMS)
- oprogramowanie służące do tworzenia i zarządzania bazami danych
- sposób trzymania danych jest zależny od konkretnego systemu

# SQL
- język służący do interakcji z __relacyjnymi bazami danych__
- (prawie) wszystkie relacyjne __DBMS__ wykorzystują __SQL__ jako podstawowy język zapytań

# Przykłady relacyjnych systemów zarządzania bazą danych
- PostgreSQL (open source)
- MySQL (open source)
- SQLite (open source)
- Oracle
- Microsoft SQL Server
- IBM DB2
- Microsoft Access

# SQLite
- nie ma prawie żadnych zależności, działa na każdym systemie operacyjnym
- nie ma potrzeby uruchamiać żadnego serwera bazy danych
- nie wymaga żadnej dodatkowej konfiguracji
- najczęście używana baza danych na świecie https://www.sqlite.org/mostdeployed.html
- cała baza danych zapisana jest w jednym pliku (cross-platform)
- obsługuje transakcje

# Jak wygląda SQLowa baza danych, na przykladzie Chinook DB
![sqlite-sample-database-color.jpg](http://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

# Tabela
- w bazie danych obiekt, który zawiera dane
- tabela składa się z wierszy i kolumn
- każda kolumna reprezentuje jakąś właściwość
- każdy wiersz reprezentuje pojedynczy rekord

# Kolumna
- właściwość rekordu (np. imię, nazwisko, identyfikator itp.)

## Ma określony typ
- NULL
- INTEGER
- REAL
- TEXT
- BLOB

# NULL
- specjalny znacznik, który mówi o tym, że dana wartość nie jest ustawiona

# PK - Primary Key
- klucz główny dla tabeli
- unikalny
- posiada index
- maksymalnie jeden

# FK - Foreign Key
- klucz obcy
- wartość z pola z rekordu z innej tabeli, do którego "linkujemy"
- musi być jednoznaczne
- bardzo często PK innej tabeli

# Index
- zakłada się go na wybranej kolumnie (lub kilku)
- specjalna struktura danych, która umożliwia szybsze przeszukiwanie danych na podstawie danej kolumny
- w SQLite (i wielu innych bazach danych) zaimplementowane na B-drzewach https://pl.wikipedia.org/wiki/B-drzewo

więcej o indeksach w SQLicie: http://www.sqlitetutorial.net/sqlite-index/

# Relacje pomiędzy tabelami
- one-to-one
- one-to-many
- many-to-many

# sqlite3 w Pythonie

```python
import sqlite3
conn = sqlite3.connect('/path/to/database.db')
conn.close()
```

więcej https://docs.python.org/3/library/sqlite3.html#connection-objects

# Chinook
Przykładowa baza danych stworzona przez zespół odpowiedzialny za SQLite

http://www.sqlitetutorial.net/sqlite-sample-database/

Pobieramy bazę: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

Następnie trzeba ją jedynie wypakować

- tworzymy obiekt __Cursor__, który umożliwia nam wykonywanie poleceń na bazie
- możemy na nim wykonywać operacje za pomocą metod __execute__ (jedna operacja), __executemany__ (jeden rodzaj operacji z wieloma różnymi parametrami) lub __executescript__ (ciąg operacji)

więcej: https://docs.python.org/3/library/sqlite3.html#cursor-objects

Tego możemy używać do przeglądania naszej bazy danych: http://sqlitebrowser.org/

# Dlaczego będziemy odpalać gołe SQL-ki?
- znajomość SQL jest kluczowa w pracy programisty
- ORM nie zawsze jest w stanie załatwić skomplikowane przypadku
- dobrze jest wiedzieć co się dzieje "pod spodem" ORMa
- znajomość SQLa ułatwia debugowanie problemów związanych z bazami danych

# Dostęp do bazy danych w aplikacji we Flasku

Połączenie do bazy danych tworzymy i trzymamy kontekście aplikacji, zamykamy połączenie przy niszczeniu kontekstu.

```python
# http://flask.pocoo.org/docs/0.12/patterns/sqlite3/
import sqlite3
from flask import g

DATABASE = '/path/to/database.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()
```

# SELECT

Pobieranie konkretnych pól z wybranej tabeli:
```sql
SELECT {lista_pól} FROM {nazwa_tabeli};
```

np.

```sql
SELECT first_name, last_name FROM actor;
```

Pobieranie wszystkich pól z wybranej tabeli:
```sql
SELECT * FROM {nazwa_tabeli};
```

np.

```sql
SELECT * FROM actor;
```

## Przykład 1
Stworzyć widok listy tytułów piosenek `/tracks`, z wykorzystaniem szablonu:
```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Tracks</title>
</head>
<body>
  <ul>
    Nazwa
    {% for track in tracks %}
      <li>{{ track[0] }}</li>
    {% endfor %}
  </ul>
</body>
</html>
```

Wszystkie przykłady są zdeployowane tu: https://da-python-levelup-2019.herokuapp.com/

```python
@app.route('/tracks')
def tracks_list():
    db = get_db()
    cursor = db.cursor()
    data = cursor.execute('SELECT name FROM tracks').fetchall()
    cursor.close()
    return render_template('tracks.html', tracks=data)
```

# Pobieranie wyników zapytań
- __fetchall__ - zwraca listę wyników 
- __fetchone__ - zwraca jeden (następny) wynik, jeżeli nie ma to `None`
- __fetchmany__(size=cursor.arraysize) - zwraca listę max `size` wyników, w przypadku braku wyników pusta lista


- domyślny cursor.arraysize to 1

# Typy w wyniku
- domyślnie wiersz przedstawiony jest jako `tuple` wartości poszczególnych kolumn
- kolumny "zachowują" swój typ z bazy danych zgodnie z tabelą https://docs.python.org/3/library/sqlite3.html#sqlite-and-python-types

Do wyników możemy dostać się też w wygodniejszy sposób, wykorzystując nazwy kolumn

ustawiamy na połączeniu `row_factory`
```python
db.row_factory = sqlite3.Row
```

Dzięki temu do tytułu filmu możemy dostać się za pomocą `film['title']` zamiast `film[0]`

Możliwe jest też zdefiniowanie własnych `row_factory` https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.row_factory

Możemy też skorzystać ze "skrótu" - odpalić `execute` od razu na `Connection`, bez tworzenia wprost obiektu `Cursora`

```python
@app.route('/tracks')
def tracks_list():
    db = get_db()
    data = db.execute('SELECT name FROM tracks').fetchall()
    return render_template('tracks.html', tracks=data)
```

# SELECT ... WHERE

Filtrowanie :
```sql
SELECT {lista_pól} FROM {nazwa_tabeli} WHERE {warunek};
```

np.

```sql
SELECT first_name, last_name FROM actor WHERE id = 1;
```

więcej: http://www.sqlitetutorial.net/sqlite-where/

## Przykład 2
Stworzyć widok pojedynczego utworu`/tracks/<int:track_id>`, z wykorzystaniem szablonu:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Tracks</title>
</head>
<body>
    <h1>{{ track['name'] }}</h1>
    <h2>Wykonawca: {{ track['composer'] }}</h2>
</body>
</html>
```

```python
@app.route('/tracks/<int:track_id>')
def single_track(track_id):
    db = get_db()
    data = db.execute(
        'SELECT name, composer FROM film WHERE track_id = ?',
        (track_id,)).fetchone()

    return render_template('single_track.html', film=data)
```

# Wstawianie zmiennych do zapytania
- znaki zapytania to placeholdery, pod które są podstawiane wartości z tupli przekazanej jako drugi argument
- `?` placeholder "pozycyjny" - pod kolejne znaki zapytania podstawiane są kolejne wartości z tupli
- `:nazwa` to "keyword" placeholder, jako drugi argument to `execute` przekazujemy słownik; dane są podstawiane na podstawie kluczy ze słownika

```python
@app.route('/tracks/<int:track_id>')
def single_track(track_id):
    db = get_db()
    data = db.execute(
        'SELECT title, release_year, description FROM film WHERE track_id = :track_id',
        {'track_id': track_id}).fetchone()

    return render_template('single_track.html', film=data)
```

# JOIN
Łączenie tabel ze sobą :
```sql
SELECT {lista_pól} FROM {nazwa_tabeli}
JOIN {nazwa_innej_tabeli} ON {warunek_joina}
(opcjonalnie) WHERE {warunek};
```

np.

```sql
SELECT actor.last_name, film.title FROM actor
JOIN film_actor ON actor.actor_id = film_actor.actor_id
JOIN film ON film_actor.film_id = film.film_id;
```

Istnieją różne typy JOINów: https://stackoverflow.com/questions/17946221/sql-join-and-different-types-of-joins

## Przykład 3
Dodaj widok listy utowrów rozszerzony o wykonawcę płyty z której utów pochodzi - `/tracks_with_artist`. Szablon:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Tracks</title>
</head>
<body>
  <ul>
      Utwór, Wykonawca
  {% for track in tracks %}
      <li>{{ track["name"] }} - {{ track["artist"] }}</li>
  {% endfor %}
  </ul>
</body>
</html>
```

```python
@app.route('/tracks_with_artist')
def tracks_with_artist():
    db = get_db()
    data = db.execute('''
     SELECT tracks.name, artists.name AS artist FROM tracks
     JOIN albums ON tracks.albumid = albums.albumid
     JOIN artists ON albums.artistid = artists.artistid;
     ''').fetchall()
    return render_template('tracks_with_artists.html', tracks=data)
```

# LIMIT, OFFSET, ORDER BY 
Pobranie rekordów w określonej kolejności
```sql
SELECT {lista_pól} FROM {nazwa_tabeli} ORDER BY {nazwa_pola};
```
Np.

```sql
SELECT * FROM actor ORDER BY last_name;
```
Pobranie ograniczonej liczby rekordów
```sql
SELECT {lista_pól} FROM {nazwa_tabeli} LIMIT {liczba_rekordów};
```
Np.
```sql
SELECT * FROM actor LIMIT 10;
```
Pobranie ograniczonej liczby rekordów z pominięciem pierwszych
```sql
SELECT {lista_pól} FROM {nazwa_tabeli} LIMIT {liczba_rekordów} OFFSET {liczba_pominiętych_rekordów};
```
Np.
```sql
SELECT * FROM actor LIMIT 10 OFFSET 20;
```

http://www.sqlitetutorial.net/sqlite-limit/

## Przykład 4
Dodaj widok listy utworów z kategorią, w którym filmy posortowane są po nazwie kategorii - `/tracks_with_album_order`.

Szablon taki sam jak w __Przykładzie 3__

```python
@app.route('/tracks_with_artists_order')
def tracks_with_artists_order():
    db = get_db()
    data = db.execute('''
     SELECT tracks.name, artists.name AS artist FROM tracks
     JOIN albums ON tracks.albumid = albums.albumid
     JOIN artists ON albums.artistid = artists.artistid
     order by artists.name;
     ''').fetchall()
    return render_template('tracks_with_artists.html', tracks=data)
```

# INSERT

Wstawienie wiersza do tabeli
```sql
INSERT INTO {nazwa_tabeli} ({lista_pól) VALUES ({lista_wartości});
```

## Przykład 5
Stwórz widok dodawania nowego aktora `/artists/add` oraz widok listy wykonawców `/artists`.

Wykorzystaj szablony:
```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Wykonawcy</title>
</head>
<body>
  <ul>
    Nazwa
    {% for artist in artists %}
      <li>{{ artist['name'] }}</li>
    {% endfor %}
  </ul>
</body>
</html>
```

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Dodaj wykonawcę</title>
</head>
<body>
<form action="/artists/add", method="post">
  Wykonawca:<br>
  <input type="text" name="name"><br>
  <input type="submit" value="Dodaj">
</form>
</body>
</html>
```

# CSRF
- Uwaga! Nasz formularz nie jest zabezpieczony zgodnie z najlepszymi praktykami
- Korzystając z HTML-owych formularzy powinniśmy zabezpieczać się przed atakiem CSRF https://www.owasp.org/index.php/Cross-Site_Request_Forgery_(CSRF)
### Jak to robić we Flasku?
- Pisząc to samemu: http://flask.pocoo.org/snippets/3/
- Lub najlepiej skorzystać z gotowego rozwiązania: http://flask-wtf.readthedocs.io/en/stable/csrf.html

```python
@app.route('/artists/add', methods=['GET', 'POST'])
def add_artist_version_1():
    if request.method == 'POST':
        name = request.form['name']
        db = get_db()
        db.executescript(
            'INSERT INTO actor (name) VALUES ("{}")'
            .format(name)
        )
        db.commit()
        return redirect(url_for('artists'))
    else:
        return render_template('add_artist.html')
```

## Co z tym kodem jest nie tak?

# SQL Injection

![xkcd SQL Injection comic](https://imgs.xkcd.com/comics/exploits_of_a_mom.png )

Źródło: https://imgs.xkcd.com/comics/exploits_of_a_mom.png

https://xkcd.com/327/

```python
@app.route('/artists/add', methods=['GET', 'POST'])
def add_artist_version_2():
    if request.method == 'POST':
        name = request.form['name']
        db = get_db()
        db.executescript(
            'INSERT INTO actor (name) VALUES (?)',
            (name, )
        )
        db.commit()
        return redirect(url_for('artists'))
    else:
        return render_template('add_artist.html')
```

# UPDATE
Aktualizacja wszystkich wierszy z tabeli
```sql
UPDATE {nazwa_tabeli} SET {nazwa_pola} = {wartość_pola}; 
```

Aktualizacja wybranych wierszy
```sql
UPDATE {nazwa_tabeli} SET {nazwa_pola} = {wartość_pola} WHERE {warunek}
```

## Przykład 6
Dodaj widok edycji artysty `/artists/edit/<int:artists_id>`. Szablon:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Edytuj artystę: {{ artist['name'] }}</title>
</head>
<body>
<form action="/artists/edit/{{ artistid }}", method="post">
  Nazwa:<br>
  <input type="text" name="name"><br><br>
  <input type="submit" value="Dodaj">
</form>
</body>
</html>
```

```python
@app.route('/artists/edit/<int:artistid>', methods=['GET', 'POST'])
def edit_artists(artistid):
    if request.method == 'POST':
        name = request.form['name']
        db = get_db()
        db.execute(
            'UPDATE artists SET name = ? WHERE artistid = ?',
            (name, artistid)
        )
        db.commit()
        return redirect(url_for('artists_list'))
    else:
        db = get_db()
        artist = db.execute(
            'SELECT name from artists WHERE artistid = ?',
            (artistid,)).fetchone()
        return render_template('edit_artist.html', artist=artist, artistid=artistid)
```

# DELETE
Usunięcie wszystkiego z tabeli
```sql
DELETE FROM {nazwa_tabeli};
```

Usunięcie wybranych wierszy
```sql
DELETE FROM {nazwa_tabeli} WHERE {warunek};
```

## Przykład 7
Dodaj możliwość usunięcia wybranego aktora `/artists/delete/<int:artist_id`. Szablon: 

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Usuń wykonawcę</title>
</head>
<body>
  Nazwa: {{ artist['name'] }}<br>
<form action="/artists/delete/{{ artist_id }}" method="post">
    <input type="submit" value="Usuń">
</form>
</body>
</html>
```

```python
@app.route('/artists/delete/<int:artist_id>', methods=['GET', 'POST'])
def delete_artist(artist_id):
    if request.method == 'POST':
        db = get_db()
        db.execute('DELETE FROM artist WHERE artist_id = ?', (artist_id,))
        db.commit()
        return redirect(url_for('artists_list'))
    else:
        db = get_db()
        artist = db.execute(
            'SELECT first_name, from artists WHERE artist_id = ?',
            (artist_id,)).fetchone()
        return render_template('delete_artist.html', artist=artist, artist_id=artist_id)
```

# Funkcje agregujące
- MIN
- MAX
- SUM
- AVG
- COUNT
- COUNT(*)

Uruchomienie funkcji agregującej dla wybranej kolumny
```sql
SELECT {funkcja_agregująca}({kolumna}) FROM {tabela};
```
Np.
```sql
SELECT MIN(actor_id) FROM actor;
```

Zliczenie wszystkich wierszy w tabeli
```sql
SELECT COUNT(*) FROM {tabela};
```
Np.
```sql
SELECT COUNT(*) FROM actor;
```

## Przykład 8
Dodaj podsumowanie ilości artystów na widoku listy artystów. Szablon: 

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Wykonawcy</title>
</head>
<body>
  Ilość Artystów w bazie: {{ count }}
  <ul>
    Nazwa
    {% for artist in artists %}
      <li>{{ artist['name'] }}</li>
    {% endfor %}
  </ul>
</body>
</html>
```

```python
@app.route('/artists_count')
def artists_list_with_count():
    db = get_db()
    data = db.execute('SELECT name FROM artists ORDER BY name DESC').fetchall()
    count = db.execute('SELECT COUNT(*) FROM artists').fetchone()
    return render_template('artists_count.html', artists=data, count=count[0])
```

# Porównanie wydajności SQL-owego AVG z obliczeniami w Pythonie

```python
import timeit

result_sql = timeit.timeit(
    "c.execute('SELECT AVG(value) FROM test_table').fetchone()",
    setup="import sqlite3;conn = sqlite3.connect('test_single_table.db');c = conn.cursor();",
    number=1
)
print(result_sql)

result_python = timeit.timeit(
    "result = c.execute('SELECT value FROM test_table').fetchall(); l = [r[0] for r in result]; sum(l) / len(l)",
    setup="import sqlite3;conn = sqlite3.connect('test_single_table.db');c = conn.cursor();",
    number=1
)
print(result_python)
```

Wynik:
```
0.073508342000423
0.7799752800056012
```

# Deploy na Heroku
- __SQLite__ na __Heroku__? Nie jest to najlepszy pomysł https://devcenter.heroku.com/articles/sqlite3
- Wystarczy wypushować dodatkowo plik z bazą danych na Heroku
- Problem: Wprowadzane w nim zmiany będą wymazywane, przynajmniej raz na 24h - po każdym deployu, restarcie i co około 24h z powodu tego , jak Heroku zarządza swoimi `dynos`
- Co z tym robimy? Na tych zajęciach nic
- Dobre rozwiązanie? Korzystanie z __Heroku Postgres__ https://elements.heroku.com/addons/heroku-postgresql zamiast z __sqlite__ ale to już na kolejnych zajęciach

# CREATE TABLE
Tworzenie tabeli
```sql
CREATE TABLE {nazwa_tabeli} {lista_pól}
```

Np.

```sql
CREATE TABLE actor (
  actor_id numeric NOT NULL ,
  first_name VARCHAR(45) NOT NULL,
  last_name VARCHAR(45) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (actor_id)
);
```

https://www.sqlite.org/lang_createtable.html

# CREATE INDEX
Tworzenie indeksu
```sql
CREATE  INDEX {nazwa_indeksu} ON {nazwa_tabeli}({nazwa_pola});
```
Np.
```sql
CREATE  INDEX idx_actor_last_name ON actor(last_name);
```

https://www.sqlite.org/lang_createindex.html

# ALTER TABLE

Wprowadzenie zmian w istniejącej tabeli. Np. dodawanie nowej kolumny.

Usunięcie tabeli
```sql
ALTER TABLE {nazwa_tabeli} {zmiana};
```
np.
```sql
ALTER TABLE actor ADD COLUMN middle_name VARCHAR(45);
```

https://www.sqlite.org/lang_altertable.html

# DROP TABLE

Usunięcie tabeli
```sql
DROP TABLE {nazwa_tabeli};
```

np.
```sql
DROP TABLE actor;
```

https://www.sqlite.org/lang_droptable.html

# EXTRA

### Sakila - Inna baz danych stworzona do testowania
Przykładowa baza danych stworzona przez zespół odpowiedzialny za MySQL, jest dużo większa od Chinook

https://dev.mysql.com/doc/sakila/en/sakila-introduction.html

Pobieramy to: https://github.com/jOOQ/jOOQ/raw/master/jOOQ-examples/Sakila/sqlite-sakila-db/sqlite-sakila-schema.sql

I to: https://github.com/jOOQ/jOOQ/raw/master/jOOQ-examples/Sakila/sqlite-sakila-db/sqlite-sakila-insert-data.sql

a następnie wykonujemy:

In [ ]:
import sqlite3

conn = sqlite3.connect('/path/to/database.db')
c = conn.cursor()

with open('sqlite-sakila-schema.sql', 'r', encoding='utf-8') as create_file:
    create_query = create_file.read()
with open('sqlite-sakila-insert-data.sql', 'r', encoding='utf-8') as insert_file:
    insert_query = insert_file.read()

c.executescript(create_query)
c.executescript(insert_query)

conn.commit()
conn.close()

- Skrypt z insertami wolno działa? http://www.sqlite.org/faq.html#q19
- Rozwiązanie: http://www.sqlite.org/lang_transaction.html


- bez BEGIN TRANSACTION; .. COMMIT; - 2h10min
- z BEGIN TRANSACTION; .. COMMIT; - kilkanaście sekund

### Tutoriale
- Interaktywny webowy tutorial SQL: https://sqlbolt.com/